In [1]:
import pandas as pd
import numpy as np
import nltk
import copy
from nltk.stem.lancaster import LancasterStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("Crowler/booking_reviews.csv")

In [3]:
data[:5]

,title,pos_text,neg_text,ratingValue,bestRating,rate
0,наступної мандрівки до Львова оберемо цей готель.,Відмінне розташування готелю! Дуже близько до ...,"один маленький мінус, але нам це не завдало не...",10.0,10.0,1
1,"Сніданок - великі порціїї, смачно!","Сніданок - великі порціїї, смачно! 1 страва бе...",Сам готель усередині нагадує гуртожиток чи хос...,6.3,10.0,0
2,"Все чудово! За нагоди, обов'язково знову завіт...",Дуже чудовий і затишний отель! Дуже зручне роз...,"Сантехніку (труби , стоки) необхідно перевірит...",9.6,10.0,1
3,"класне розташування, такий собі острівець тиші...","класне розташування, такий собі острівець тиші...",начебто новий ремонт - а велика тріщина над дв...,7.5,10.0,0
4,"Гарне місце для туристів або відрядження, але ...","Ідеальний номер, як для тризіркового готелю - ...",Дуже круті сходи. Складно знайти поруч місце д...,9.2,10.0,1


In [4]:
def read_stop_words(file):
    with open(file) as f:
        stop_words = f.read().split('\n')

    return stop_words

In [5]:
stop_words = read_stop_words('ukrainian-stopwords.txt')

In [6]:
ignore_symbols = [',', '.', '!', '?']

In [7]:
ignore = ignore_symbols + stop_words

In [8]:
stemmer = LancasterStemmer()

In [9]:
def add_words_from_sentence_to_bag_of_words(sentence : str, ignore_words: list, bag_of_words : set):
    words = nltk.word_tokenize(sentence)
    for word in words:
        if word in ignore_words:
            continue
        bag_of_words.add(stemmer.stem(word.lower()))

In [10]:
bag_of_words = set()
for i in range(0, len(data)):
    add_words_from_sentence_to_bag_of_words(data['title'][i], ignore, bag_of_words)
    add_words_from_sentence_to_bag_of_words(data['pos_text'][i], ignore, bag_of_words)
    add_words_from_sentence_to_bag_of_words(data['neg_text'][i], ignore, bag_of_words)

In [11]:
def create_bag_of_words_vector(sentence: str, ignore_words : list, bag_of_words : set):
    d = dict.fromkeys(bag_of_words, 0) 
    words = nltk.word_tokenize(sentence)
    for word in words:
        if word in d:
            d[word] += 1
            
    return list(d.values())

In [12]:
bag_of_words_dict = {}
for word in bag_of_words:
    bag_of_words_dict[word] = len(data) * [0]
    

In [13]:
# bag_of_words_dict = dict.fromkeys(bag_of_words, copy.copy(zeros)) 

In [14]:
def add_item_to_bag_of_words_dict(sentence: str, ignore_words : list, index, bag_of_words_dict : dict):
    words = nltk.word_tokenize(sentence)
    for word in words:
        if word in bag_of_words_dict:
            bag_of_words_dict[word][index] += 1

In [15]:
X, y = [], []
for i in range(0, len(data)):
    title = create_bag_of_words_vector(data['title'][i], ignore, bag_of_words)
    pos_text = create_bag_of_words_vector(data['pos_text'][i], ignore, bag_of_words)
    neg_text = create_bag_of_words_vector(data['neg_text'][i], ignore, bag_of_words)
    rate_value = data['rate'][i]
    xi = title + pos_text + neg_text
    xi = np.array(title) + np.array(pos_text) + np.array(neg_text)
    yi = rate_value
    
    X.append(xi)
    y.append(yi)

In [16]:
y = []
for i in range(0, len(data)):
    add_item_to_bag_of_words_dict(data['title'][i], ignore, i, bag_of_words_dict)
    add_item_to_bag_of_words_dict(data['pos_text'][i], ignore, i, bag_of_words_dict)
    add_item_to_bag_of_words_dict(data['neg_text'][i], ignore, i, bag_of_words_dict)

    rate_value = data['rate'][i]
    yi = rate_value
    
    y.append(yi)

In [17]:
df = pd.DataFrame.from_dict(bag_of_words_dict)

In [22]:
df[:5]

,решт,смешные,кондиционер,послизнутися,локація！,некомфортність,висока,щіток,великою,просочувалась,...,однаковий,тут,клітки,засобом,руху,попередження,чула,пустий,приехал,вияснити
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [19]:
clf = LogisticRegression(random_state=42, solver='lbfgs', multi_class='multinomial')

In [20]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [21]:
len(X_train)

825

### Metric 

In [23]:
from sklearn.metrics import classification_report

In [24]:
y_pred = clf.predict(X_test) 

In [25]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.50      0.16      0.24        19
           0       0.56      0.58      0.57        77
           1       0.65      0.70      0.68       111

   micro avg       0.61      0.61      0.61       207
   macro avg       0.57      0.48      0.49       207
weighted avg       0.60      0.61      0.60       207



### Feature importance

#### Lime

In [26]:
len(X_train.columns)

8770

Count of some specific words in all dataset

In [34]:
def print_num_of_word_occurrences(word):
    print(f"{word} - {sum(bag_of_words_dict[word])}")

In [37]:
print_num_of_word_occurrences("шикарне")
print_num_of_word_occurrences("поверхню")
print_num_of_word_occurrences("😁😉")

шикарне - 1
поверхню - 1
😁😉 - 1


In [ ]:
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values,
                                                   mode = 'classification',
                                                   feature_names = X_train.columns,
                                                   categorical_features = [], 
                                                   categorical_names = [], 
                                                   discretize_continuous = True)
                                                   
np.random.seed(42)
exp = explainer.explain_instance(X_train.values[31], clf.predict_proba, num_features = len(X_train.columns))
exp.show_in_notebook(show_all=False) #only the features used in the explanation are displayed



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [38]:
sum(bag_of_words_dict['болить'])

1

In [39]:
sum(bag_of_words_dict['готель'])

227